# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
%pip install --upgrade pip --quiet --no-color
%pip install anthropic --force-reinstall --quiet --no-color

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 6.1.0 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
scikit-image 0.24.0 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
sphinx 8.0.2 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [2]:
modelId

'anthropic.claude-3-haiku-20240307-v1:0'

In [6]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 200,  # Set the maximum number of tokens to generate
        "topP": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]

    # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

Amazon Bedrock offers three APIs that can be used with Anthropic Claude models, the legacy [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) , the [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) and the current [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). For this tutorial, we will be exclusively using the Converse API.

At minimum, a call to Claude using the Converse API requires the following parameters:
- `modelId`: the [API model name](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns) of the model that you intend to call

- `messages`: an array of input messages. Claude 3 models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). **The first message must always use the `user` role.**
  
  You store the content for the message in the `content` field of a [(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html). Specify text in the `text` field, or if supported by the model, you can also pass the raw bytes for an image in the `image` field of an [(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html). The other fields in ContentBlock are for [tool use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html).

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

For a complete list of all API parameters, visit our [API documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [3]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [4]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect onto the surface of the ocean, making it appear more gray, white, or even darker blue on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the water and the lack of


In [5]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.


Now let's take a look at some prompts that do not include the correct Converse API formatting. For these malformatted prompts, the Converse API returns an error.

First, we have an example of a Converse API call that lacks `role` and `content` fields in the `messages` array.

> ⚠️ **Warning:** Due to the incorrect formatting of the messages parameter in the prompt, the following cell will return an error. This is expected behavior.

In [3]:
# Ensure modelId is defined somewhere in your code
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'  # Replace with your actual model ID

# Configuration for inference
inference_config = {
    "temperature": 0.0,  # Set the temperature for generating diverse responses
    "maxTokens": 200     # Set the maximum number of tokens to generate
}

# Parameters for the converse API call
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": ["Hi Claude, how are you?"]}  # Content should be a list
    ],
    "inferenceConfig": inference_config  # Pass the inference configuration
}

# Making the API call
try:
    response = bedrock_client.converse(**converse_api_params)
    
    # Assuming the response structure has 'output' -> 'messages' -> list of messages -> 'content'
    # Adjust according to your actual response format
    if 'output' in response and 'messages' in response['output']:
        # Extracting the content from the first message in the response
        message_content = response['output']['messages'][0]['content']
        # Join the content list into a single string if necessary
        print(" ".join(message_content))
    else:
        print("Unexpected response format:", response)

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Parameter validation failed:
Invalid type for parameter messages[0].content[0], value: Hi Claude, how are you?, type: <class 'str'>, valid types: <class 'dict'>


Here's a prompt that fails to alternate between the `user` and `assistant` roles.

> ⚠️ **Warning:** Due to the lack of alternation between `user` and `assistant` roles, Claude will return an error message. This is expected behavior.

In [4]:
inference_config = {
    "temperature": 0.5,
    "maxTokens": 200
}
# Create the converse method parameters
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "user", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

ValidationException: An error occurred (ValidationException) when calling the Converse operation: A conversation must alternate between user and assistant roles. Make sure the conversation alternates between user and assistant roles and try again.

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. 

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook). 

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [5]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

NameError: name 'get_completion' is not defined

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit Anthropic's documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.

In [9]:
# Prompt - this is the only field you should change
# PROMPT = "[Replace this text]"
PROMPT = "[Count from one to three]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1, 2, 3.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [13]:
print(hints.exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [7]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "a 3 year old child assistant"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

The sky is very large, but it's difficult to give an exact size for it. Here are some key points about the size of the sky:

- The sky refers to the area of space above the Earth's surface. It extends as far as the Earth's atmosphere goes, which is about 6,200 miles (10,000 km) above the surface.

- Beyond the Earth's atmosphere, the sky transitions into outer space, which is essentially limitless. The observable universe is estimated to be about 93 billion light-years in diameter.

- The visible portion of the sky that we can see from Earth's surface is called the celestial sphere. This appears to be a dome or hemisphere covering the Earth, but it's actually just our perspective of a much larger expanse.

- The size of the visible sky depends on your location on Earth. From a single vantage point, the visible sky covers about 180-200 degrees horizont

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ If you want a hint, run the cell below!

In [8]:
print(hints.exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [9]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [10]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect onto the surface of the ocean, making it appear more gray, white, or even darker blue on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the water and the lack of


In [11]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.


In [13]:
# Ensure modelId is defined somewhere in your code
modelId = 'anthropic.claude-3-haiku-20240307-v1:0' # Replace with your actual model ID

# Configuration for inference
inference_config = {
    "temperature": 0.0  # Control the randomness of responses
}

# Additional fields for model request
additional_model_fields = {
    "max_tokens": 200  # Set the maximum number of tokens for the response
}

# Parameters for the converse API call
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": "Hi Claude, how are you?"}  # Correct format for messages
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

# Making the API call
try:
    response = bedrock_client.converse(**converse_api_params)
    
    # Adjust the response parsing based on the actual API response structure
    # This assumes response['output']['messages'][0]['content'] contains the text
    if 'output' in response and 'messages' in response['output']:
        message_content = response['output']['messages'][0]['content']
        print(message_content)  # Directly print the content assuming it's a string
    else:
        print("Unexpected response format:", response)

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Parameter validation failed:
Invalid type for parameter messages[0].content, value: Hi Claude, how are you?, type: <class 'str'>, valid types: <class 'list'>, <class 'tuple'>


In [19]:
# Ensure modelId is defined somewhere in your code
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'  # Replace with your actual model ID

# Configuration for inference
inference_config = {
    "temperature": 0.0
}

# Additional fields for model request
additional_model_fields = {
    "top_p": 1,
    "max_tokens": 200
}

# Parameters for the converse API call
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": "What year was Celine Dion born in?"},
        {"role": "user", "content": "Also, can you tell me some other facts about her?"}
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

# Ensure `bedrock_client` is properly initialized (replace with actual initialization)
# For example: bedrock_client = SomeBedrockClientLibrary.initialize()

# try:
#     # Making the API call
#     response = bedrock_client.converse(**converse_api_params)
    
#     # Print response content
#     # Adjust the response access according to the actual response structure
#     # Assuming the response structure: response['output']['message']['content']
    
#     if 'output' in response and 'message' in response['output'] and 'content' in response['output']['message']:
#         print(response['output']['message']['content'])
#     else:
#         print("Unexpected response structure:", response)

# except Exception as e:
#     print(f"An error occurred: {e}")

try:
    # Making the API call
    bedrock_client = SomeBedrockClientLibrary.initialize()
    response = bedrock_client.converse(**converse_api_params)
    
    
    # Print response content
    # Adjust the response access according to the actual response structure
    # Assuming the response structure: response['output']['message']['content']
    
    if 'output' in response and 'message' in response['output'] and 'content' in response['output']['message']:
        print(response['output']['message']['content'])
    else:
        print("Unexpected response structure:", response)

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: name 'SomeBedrockClientLibrary' is not defined


In [20]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What is the composition of the Earth's atmosphere, and how does that affect the way light interacts with it?
- How do different wavelengths of light behave as they pass through the atmosphere?
- What is the process known as Rayleigh scattering, and how does it contribute to the blue appearance of the sky?
- How do factors like the time of day, weather conditions, and location on Earth influence the appearance of the sky?
- What other celestial bodies exhibit different colors in their skies, and what does that tell us about their atmospheric compositions?
- How have our scientific understandings of the sky's color changed over time as our knowledge of optics and atmospheric science has advanced?

Exploring these types of questions can help deepen our understanding of the physical principles behind the sky's blue appearance. Let me know if you have any other
